## Project Name: Orcasound Salmon
### Program Name: noaa_ais_shipping_traffic.ipynb
### Purpose: To explore and visualize the shipping traffic off the west coast using NOAA AIS archives
##### Date Created: Apr 22 2023

In [14]:
import os 
from os.path import join as pjoin 
import datetime 
import pandas as pd
import pytz 
from pytz import timezone 
import numpy as np 
import transbigdata as tbd
import keplergl
from keplergl import KeplerGl

Define path and data file

In [25]:
base_path=os.path.abspath('')
data_path=pjoin(base_path, "data")
ship_fname=["AIS_2016_02_24.csv"]
srkw_data_fname="SRKW occurrence coastal - SRKW occurrence coastal Data.csv"

Processing orca data

In [50]:
srkw_data=pd.read_csv(pjoin(data_path, srkw_data_fname))
srkw_data=srkw_data.rename(columns={'Lat P':'lat_p', 'Lon P':'lon_p', 'Lat A':'lat_a','Lon A':'lon_a'})
srkw_data=srkw_data[['Animal', 'lat_p', 'lon_p', 'lat_a',
       'lon_a', 'Dur', 'Sex', 'Popid', 
       'Month', 'Day', 'Year', 'Hour', 'Minute', 'Second']]
srkw_data['Popid']=srkw_data['Popid'].apply(lambda x: x.replace(' ',''))

Convert GMT time to Pacific time

In [15]:
def gmt2pst(yr,mon,day,h,m,s):
    tz=pytz.timezone('GMT')
    _date1=datetime.datetime(yr, mon, day,h,m,s, tzinfo=tz)
    _date2=_date1.astimezone(timezone('US/Pacific'))
    return(_date2)

Add a numerical day with 0 being the day of the earliest record for each orca

In [17]:
def orcaday(dat):
    day0=min(dat['date'])
    dat['day_move']=dat['date'].apply(lambda x: x-day0)

Use alternative longitude and latitude values when the primary values are out of bound

In [18]:
def geogen(dat):
    import numpy as np
    dat['lon']=np.where(dat['lon_p']>-120, dat['lon_a'],dat['lon_p'])
    dat['lat']=np.where(dat['lon_p']>-120, dat['lat_a'],dat['lat_p'])

In [51]:
srkw_data['datetime_pst']=srkw_data.apply(lambda x: gmt2pst(x.Year, x.Month, x.Day, x.Hour, x.Minute, x.Second), axis=1)
srkw_data=srkw_data.drop(columns=['Year','Month','Day','Hour','Minute','Second'])
srkw_data['date']=srkw_data.apply(lambda x: x.datetime_pst.date(), axis=1)
srkw_data['time']=srkw_data.apply(lambda x: x.datetime_pst.time(), axis=1)
srkw_data['year']=srkw_data.apply(lambda x: x.date.year, axis=1)
srkw_data['month']=srkw_data.apply(lambda x: x.date.month, axis=1)
srkw_data['day']=srkw_data.apply(lambda x: x.date.day, axis=1)
srkw_data['hour']=srkw_data.apply(lambda x: x.time.hour, axis=1)
srkw_data['minute']=srkw_data.apply(lambda x: x.time.minute, axis=1)
srkw_data['second']=srkw_data.apply(lambda x: x.time.second, axis=1)
data_l95=srkw_data[srkw_data.Popid=='L95']
geogen(data_l95)

/var/folders/ys/_3dkbpp11vz9cw3rgqq0kj540000gn/T/ipykernel_51148/2174696341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['lon']=np.where(dat['lon_p']>-120, dat['lon_a'],dat['lon_p'])
/var/folders/ys/_3dkbpp11vz9cw3rgqq0kj540000gn/T/ipykernel_51148/2174696341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['lat']=np.where(dat['lon_p']>-120, dat['lat_a'],dat['lat_p'])


In [52]:
data_l95['id']='L95'

/var/folders/ys/_3dkbpp11vz9cw3rgqq0kj540000gn/T/ipykernel_51148/2160943547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l95['id']='L95'


Find out long and lat range for L95

In [61]:
lat_min=min(data_l95['lat'])
lat_max=max(data_l95['lat'])
lon_min=min(data_l95['lon'])
lon_max=max(data_l95['lon'])
print('min lat: ', lat_min, 'max lat: ', lat_max)
print('min lon: ', lon_min, 'max lon: ', lon_max)

min lat:  46.141 max lat:  47.517
min lon:  -124.779 max lon:  -124.074


Processing traffic data

FAQ about the data: https://coast.noaa.gov/data/marinecadastre/ais/faq.pdf 

In [ ]:
ship160224_data=pd.read_csv(pjoin(data_path, ship_fname[0]))

In [38]:
ship160224_data['datetime_utc']=ship160224_data['BaseDateTime'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))

In [43]:
ship160224_data['datetime_pst']=ship160224_data['datetime_utc'].apply(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))

Check date conversion

In [48]:
print(ship160224_data['BaseDateTime'][1])
print(ship160224_data['datetime_utc'][1])
print(ship160224_data['datetime_pst'][1])

2016-02-24T00:01:04
2016-02-24 00:01:04
2016-02-23 16:01:04-08:00


In [63]:
ship160224_data=ship160224_data.rename(columns={'LON':'lon','LAT':'lat', 'VesselName':'id'})
ship160224_crop=ship160224_data[(ship160224_data.lat<=lat_max) & (ship160224_data.lat>=lat_min) & (ship160224_data.lon<=lon_max) & (ship160224_data.lon>=lon_min)]

In [71]:
ship_orca=pd.concat([ship160224_crop[['lon','lat','datetime_pst','id']], data_l95[['lon','lat','datetime_pst','id']]])

In [75]:
ship_orca.to_csv(pjoin(data_path, "shiporca_20160224.csv"), index=False)

L95 movement

In [72]:
tbd.visualization_trip(data_l95, col=['lon', 'lat', 'id', 'datetime_pst'],)

Processing trajectory data...
Generate visualization...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hizm36i', 'type': …

L95+Shipps movement

In [73]:
tbd.visualization_trip(ship_orca, col=['lon', 'lat', 'id', 'datetime_pst'],)

Processing trajectory data...
Generate visualization...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hizm36i', 'type': …